In [1]:
%%capture
import kagglehub
path = kagglehub.model_download("deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b")

In [2]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/1


In [3]:
!pip install opendatasets

In [4]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/artemgoncarov/math-problems-imo")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/artemgoncarov/math-problems-imo


100%|██████████| 63.0M/63.0M [00:00<00:00, 153MB/s]


In [5]:
!mkdir offline_packages

In [6]:
%%writefile requirements.txt
git+https://github.com/huggingface/trl.git
peft>=0.14.0
bitsandbytes>=0.45.0
levenshtein>=0.26.1
setuptools>=40.8.0
vllm>=0.7.1

Writing requirements.txt


In [7]:
! python -m pip download --destination-directory ./offline_packages -r requirements.txt

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-kh6m77dx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-kh6m77dx
  Resolved https://github.com/huggingface/trl.git to commit af4ad47035529164799be10f3fe558ee642a9880
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━

In [8]:
!python -m pip install --no-index -v --find-links=/content/offline_packages trl --pre

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Looking in links: /content/offline_packages
Processing ./offline_packages/trl-0.15.0.dev0.zip
  Running command pip subprocess to install build dependencies
  Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Non-user install by explicit request
  Ignoring indexes: https://pypi.org/simple
  Created build tracker: /tmp/pip-build-tracker-dsbzvfxn
  Entered build tracker: /tmp/pip-build-tracker-dsbzvfxn
  Created temporary directory: /tmp/pip-install-b7lcuu2e
  Created temporary directory: /tmp/pip-ephem-wheel-cache-8cndc47d
  Looking in links: /content/offline_packages
  0 location(s) to search for versions of setuptools:
  Found link file:///content/offline_packages/setuptools-75.8.0-py3-none-any.whl, version: 75.8.0
  Local files found: /content/offline_packages/setuptools-75.8.0-py3-none-any.whl
  Given no hashes to check 1 links for project 'setuptools': discarding no candi

In [10]:
!python -m pip install --no-index -v --find-links=/content/offline_packages levenshtein

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Looking in links: /content/offline_packages
Processing ./offline_packages/levenshtein-0.26.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing ./offline_packages/rapidfuzz-3.12.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from levenshtein)


In [9]:
!python -m pip install --no-index -v --find-links=/content/offline_packages -U bitsandbytes

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Looking in links: /content/offline_packages
Processing ./offline_packages/bitsandbytes-0.45.1-py3-none-manylinux_2_24_x86_64.whl


In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [12]:
from datasets import load_dataset,Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import GRPOConfig, GRPOTrainer

import datetime

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig,
    PrinterCallback,
)
from tqdm import tqdm
import torch
import time
import transformers
import pandas as pd
import numpy as np

from Levenshtein import ratio as levenshtein_ratio
transformers.set_seed(42)

In [13]:
class CFG:
    MAX_TRAIN = 100
    MAX_TOKENS = 2048
    NUM_GENERATIONS = 4
    USE_PEFT = True
    BATCH_SIZE=1
    MAX_STEPS = 80

    BETA = 0.04
    LR = 1.e-5

    model_name = '/root/.cache/kagglehub/models/deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/1'
    splitter = '<｜Assistant｜>'

    step_count=10
    DEBUG = False

In [14]:
import re

def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""

In [15]:
df = pd.read_parquet('/content/math-problems-imo/math_problems.parquet')
df = df.reset_index().rename({'index': 'id'}, axis=1)
df['answer'] = df['solution'].map(extract_boxed_text)

def is_valid_answer(s):
    try:
        if float(s) == int(s):
            i = int(s)
            return 0<=i<1000
        else:
            return False
    except ValueError:
        return False

mask = df['answer'].map(is_valid_answer)
df = df[mask]

In [16]:
df = df.iloc[:CFG.MAX_TRAIN]

In [17]:
dataset = Dataset.from_pandas(df)

In [18]:
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'problem', 'solution', 'answer', '__index_level_0__'],
        num_rows: 90
    })
    test: Dataset({
        features: ['id', 'problem', 'solution', 'answer', '__index_level_0__'],
        num_rows: 10
    })
})

In [19]:
def create_prompt(sample):
    question = sample['problem']
    chat = [{"role": "system", "content": "A conversation between User and Assistant. The user asks a question, and the Assistant solves it.  The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>"},
            {"role": "user", "content": question + ' Return final answer within \\boxed{}, after taking modulo 1000.'},]
    sample['prompt'] = tokenizer.apply_chat_template(
            conversation=chat,
            tokenize=False,
            add_generation_prompt=True
        )
    return sample

In [20]:
dataset['train'][0]

{'id': 8,
 'problem': 'Given a set of data $x_1, x_2, x_3, x_4, x_5$ with a mean of 8 and variance of 2, find the mean and variance of a new set of data: $4x_1+1, 4x_2+1, 4x_3+1, 4x_4+1, 4x_5+1$.',
 'solution': "Firstly, let's focus on the mean of the new set. Since the original set has a mean of 8, when we apply the transformation $y_i = 4x_i + 1$ to each element, the mean will be scaled by a factor of 4 and then increased by 1. Therefore, the mean of the new set is given by:\n\\[\n\\text{New mean} = 4 \\cdot \\text{Mean of original set} + 1 = 4 \\cdot 8 + 1 = 32 + 1 = \\boxed{33}.\n\\]\n\nNext, we consider the change in variance due to this transformation. Adding 1, a constant, does not affect the variance. However, multiplying by 4 scales the variance by the square of that factor. Therefore, the variance of the new set is given by:\n\\[\n\\text{New variance} = 4^2 \\cdot \\text{Variance of original set} = 16 \\cdot 2 = \\boxed{32}.\n\\]\n\nHence, the correct answer for the mean and 

In [21]:
import re

def format_reward_func(completions, **kwargs):
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<think>.*?</think>.*?oxed{(.*?)}.*?$"
    matches = [re.match(pattern, content, re.DOTALL) for content in completions]
    return [1.0 if match else 0.0 for match in matches]


def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""

def accuracy_reward_func(completions, answer, **kwargs):
    contents = [extract_boxed_text(completion) for completion in completions]
    return [1.0 if c == str(gt) else 0.0 for c, gt in zip(contents, answer)]

In [22]:
def levenshtein_reward_func(completions, solution, **kwargs):
    res = []
    for completion, sol in zip(completions, solution):
        if '</think>' in completion:
            t = completion.split('</think>')[-1]
            res.append(levenshtein_ratio(t, sol))
        else:
            res.append(0.0)
    return res

In [23]:
device_map = 'auto'
if CFG.USE_PEFT:
    compute_dtype = getattr(torch, "float16")
    bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=False,
        )
    original_model = AutoModelForCausalLM.from_pretrained(CFG.model_name,
                                                          device_map=device_map,
                                                          quantization_config=bnb_config,
                                                          trust_remote_code=True)
else:
    original_model = AutoModelForCausalLM.from_pretrained(CFG.model_name,
                                                          device_map=device_map,
                                                          trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name,trust_remote_code=True,padding_side="left")

In [25]:
dataset = dataset.map(create_prompt)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [26]:
def gen(model, text, max_tokens):
    model_input = tokenizer(text, return_tensors='pt').to(model.device)
    model.eval()
    with torch.no_grad():
        tok = model.generate(**model_input, max_new_tokens=max_tokens, pad_token_id=tokenizer.pad_token_type_id)
        outputs = []
        for i in range(len(tok)):
            res = tokenizer.decode(tok[i], skip_special_tokens=True)
            output = res.split(CFG.splitter)[-1]
            outputs.append(output)
        return outputs[0] if len(outputs) == 1 else outputs

In [27]:
def evaluate_rewards(model, dataset, reward_functions: dict[str, callable], max_tokens: int, num_generations: int):
    completions = []
    other_info = []
    for example in tqdm(dataset):
        txt = example['prompt']
        kw = {k: v for k, v in example.items() if k not in {'prompt', 'completion'}}
        for _ in range(num_generations):
            other_info.append(kw)

        completion = gen(model, [txt]*num_generations, max_tokens)
        if isinstance(completion, str):
            completions.append(completion)
        else:
            completions += completion

    kwargs = {k: [d[k] for d in other_info] for k in other_info[0].keys()}
    res = {}
    for nm, reward_func in reward_functions.items():
        v = reward_func(completions=completions, **kwargs)
        print(nm, np.mean(v))
        res[nm] = np.mean(v)
    return res

In [28]:
reward_functions = {'formatting': format_reward_func, 'accuracy': accuracy_reward_func, 'solution_quality': levenshtein_reward_func}

In [29]:
if not CFG.DEBUG:
    original_rewards = evaluate_rewards(model=original_model, dataset=dataset['test'], reward_functions=reward_functions, max_tokens=CFG.MAX_TOKENS, num_generations=CFG.NUM_GENERATIONS)

100%|██████████| 10/10 [11:54<00:00, 71.48s/it]

formatting 0.9
accuracy 0.75
solution_quality 0.3637812904783056


In [30]:
dtstr = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
output_directory=f"./DEEPSEEK-GRPO-{dtstr}"

training_args = GRPOConfig(
    output_dir=output_directory,

    learning_rate=CFG.LR,

    per_device_train_batch_size=CFG.BATCH_SIZE,

    gradient_accumulation_steps=1,
    max_steps=CFG.MAX_STEPS,

    max_completion_length=CFG.MAX_TOKENS,
    num_generations=CFG.NUM_GENERATIONS,
    beta=CFG.BETA,

    logging_steps=CFG.step_count,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=CFG.step_count,
    gradient_checkpointing=True,
    report_to="wandb",
    overwrite_output_dir = 'True',
)

In [31]:
if CFG.USE_PEFT:
    peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        target_modules=[
            'q_proj',
            'k_proj',
            'v_proj',
            'dense'
        ],
        bias="none",
        lora_dropout=0.05,
        task_type="CAUSAL_LM",
    )
    trainer = GRPOTrainer(
        model=original_model,
        reward_funcs=list(reward_functions.values()),
        args=training_args,
        train_dataset=dataset['train'],
        peft_config=peft_config,
        callbacks=[PrinterCallback()]
    )
else:
    trainer = GRPOTrainer(
        model=original_model,
        reward_funcs=list(reward_functions.values()),
        args=training_args,
        train_dataset=dataset['train'],
        callbacks=[PrinterCallback()]
    )

In [ ]:
trainer.train()

In [ ]:
if CFG.USE_PEFT:
    print('Loading trained model')
    CHKPT = CFG.MAX_STEPS
    adapter_model_name = f'{output_directory}/checkpoint-{CHKPT}/'
    new_model = PeftModel.from_pretrained(original_model, adapter_model_name)
else:
    new_model = original_model

In [ ]:
rewards = evaluate_rewards(model=new_model, dataset=dataset['test'], reward_functions=reward_functions, max_tokens=CFG.MAX_TOKENS, num_generations=CFG.NUM_GENERATIONS)
rewards

In [35]:
print("hello world")

hello world
